In [1]:
# Import packages
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
# Set environment
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0" #Only use GPU 0
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}

In [3]:
# Congifures
ticker = "510050.SH"
pastDays = 400
lookbackNumWindowsForSingals = 20

output_dir = "./"
featureDir = "./Features/"
readInCols = ["Date", "VWAP", "todayOpen", "zt", "zt0", "zt1", "zt2", "zt3", "zt4"]
cols = ['ticker', 'date', 'delta', 'nextCloseToOpen', 'nextDayOpen','TR', 'SR', 
        'nDCtoCPnL', 'nDCtoCCumuPnL', 'nDCtoCPnLMinusTC', 'nDCtoCCumuPnLMinusTC']

startIdx = 4
n_steps = 20
n_inputs = 5
n_hidden_1 = 128
n_hidden_2 = 128
n_hidden_3 = 128
n_hidden_4 = 16
n_neurons = 1
n_outputs = 1

n_epochs = 200
batch_size = 200

c = 0.0002
learning_rate = 0.001

In [4]:
# Defind reset function for neural network
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [5]:
# Load feature data
features = {}
features[ticker] = pd.read_csv(featureDir + "features_" + ticker + ".csv", 
                               usecols=readInCols)

In [6]:
# Load date information
featureData = features[ticker]
dates = featureData.Date
date_index = np.unique(dates)
date_num = date_index.size

In [7]:
# Define empty output file dataframe
outputFile = pd.DataFrame(columns=cols)

In [9]:
# Initalize trading PnL
cumuPnL = 0
cumuPnLMinusTC = 0

# Go through All the date
for i in range(date_num - pastDays - 1):
    if i == 1:
        break
    
    # Get the train data and the next day data
    dataPd = featureData[i: i + pastDays]
    dataNextD = featureData[i + pastDays: i + pastDays + 1] 
    
    # Get features, zt0, and todayOpen
    F_train = dataPd.iloc[:int(len(dataPd)/n_steps) * n_steps, 
                          startIdx:(n_inputs + startIdx)].values
    z_train = dataPd.zt.values[:int(len(dataPd)/n_steps) * n_steps]
    p_train= dataPd.todayOpen.values[:int(len(dataPd)/n_steps) * n_steps]
    
    reset_graph()
    
    # Build data structures for f, z and p
    f = tf.placeholder(tf.float32, shape=(None, n_inputs), name="input")
    z = tf.placeholder(tf.float32, shape=(None, n_steps, n_outputs), name="z")
    p = tf.placeholder(tf.float32, shape=(None, n_steps, n_outputs), name="p")
    
    # Build neural network layers
    with tf.name_scope("DNN"):
        hidden_1 = tf.layers.dense(f, n_hidden_1, activation=tf.nn.selu, name="hidden1")
        hidden_2 = tf.layers.dense(hidden_1, n_hidden_2, activation=tf.nn.selu, name="hidden2")
        hidden_3 = tf.layers.dense(hidden_2, n_hidden_3, activation=tf.nn.selu, name="hidden3")
        hidden_4 = tf.layers.dense(hidden_3, n_hidden_4, activation=tf.nn.selu, name="hidden4")

    # Define the final delta (position)
    F = tf.reshape(hidden_4, [-1, n_steps, n_hidden_4])
    cell = tf.contrib.rnn.OutputProjectionWrapper(
            tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons, activation=None, name="rnn"), 
            output_size=n_outputs)
    
    deltaTemp, states = tf.nn.dynamic_rnn(cell, F, dtype=tf.float32)
    delta = tf.nn.relu(deltaTemp, name="deltCalc")
    
    # Define the result of the return every day (R) and the total return (U)
    print(z)
    R = tf.pad(delta[:, :(n_steps-1), :] * z[:, 1:(n_steps), :] 
               - tf.abs(delta[:, 1:n_steps, :] - delta[:, :(n_steps-1), :]) 
               * p[:, 1:(n_steps)] * c, paddings=[[0, 0], [1, 0], [0, 0]])
    U = tf.reduce_mean(tf.reduce_sum(R, axis=1))
    
    # Define an optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    
    comp_grad = optimizer.compute_gradients(-U)
    
    optimizer_gradient = optimizer.apply_gradients(comp_grad)
    
    # Define total return and sharpe ratio
    TR = tf.reduce_sum(R)
    SR = tf.reduce_mean(R) / (tf.sqrt(tf.nn.moments(tf.reshape(R, [-1]), 
                                                    axes=0)[1]) + 1e-10) * np.sqrt(252)
    
    # Initialize and define Saver
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
    # Train and test the model
    with tf.Session() as sess:
        init.run()
        for epoch in range(n_epochs):
            for iteration in range(F_train.shape[0] // batch_size):
                f_batch = F_train[iteration*batch_size: (iteration+1)*batch_size, :]
                z_batch = z_train[iteration*batch_size:(iteration+1)*batch_size]
                p_batch = p_train[iteration*batch_size:(iteration+1)*batch_size]
                z_batch = z_batch.reshape((-1, n_steps, n_outputs))
                p_batch = p_batch.reshape((-1, n_steps, n_outputs))
                sess.run(optimizer_gradient, feed_dict={f: f_batch, z: z_batch, p: p_batch})

        z_input = z_train.reshape((-1, n_steps, n_outputs))
        p_input = p_train.reshape((-1, n_steps, n_outputs))

        delta_sanityCheck = delta.eval(feed_dict={f: F_train, z: z_input, p:p_input}).flatten()
        U_sanityCheck = U.eval(feed_dict={f: F_train, z: z_input, p: p_input}).flatten()
        TR_sanityCheck = SR.eval(feed_dict={f: F_train, z: z_input, p: p_input}).flatten()
        SR_sanityCheck = SR.eval(feed_dict={f: F_train, z: z_input, p: p_input}).flatten()
        nextDayPnL = delta_sanityCheck.flatten()[-1] * dataNextD["zt"].values[-1]
        
        if i == 0:
            nextDayPnLMinusTC = nextDayPnL - np.abs(delta_sanityCheck.flatten()[-1]) * c
        else:
            nextDayPnLMinusTC = nextDayPnL - np.abs(delta_sanityCheck.flatten()[-1] - delta_sanityCheck.flatten()[-2]) * c
            
        cumuPnLMinusTC += nextDayPnLMinusTC
        resultForThisDay = [ticker, str(date_index[i+pastDays]), 
                            delta_sanityCheck.flatten()[-1], dataNextD["zt"].values[-1], 
                            dataNextD["todayOpen"].values[-1], TR_sanityCheck[-1], 
                            SR_sanityCheck[-1], nextDayPnL, cumuPnL, nextDayPnLMinusTC, 
                            cumuPnLMinusTC]
        outputFile = outputFile.append(pd.DataFrame(data=[resultForThisDay], columns=cols))
        # print(resultForThisDay)

Tensor("z:0", shape=(?, 20, 1), dtype=float32)
